In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

[![View on GitHub][github-badge]][github-keras-v3] [![Open In Colab][colab-badge]][colab-keras-v3] [![Open in Binder][binder-badge]][binder-keras-v3]

[github-badge]: https://img.shields.io/badge/View-on%20GitHub-blue?logo=GitHub
[colab-badge]: https://colab.research.google.com/assets/colab-badge.svg
[binder-badge]: https://static.mybinder.org/badge_logo.svg

[github-keras-v3]: https://github.com/mbrukman/reimplementing-ml-papers/blob/main/lenet/LeNet_Keras_v3_Subsamping_fixed_scaling_and_learning_rate_decay.ipynb
[colab-keras-v3]: https://colab.research.google.com/github/mbrukman/reimplementing-ml-papers/blob/main/lenet/LeNet_Keras_v3_Subsamping_fixed_scaling_and_learning_rate_decay.ipynb
[binder-keras-v3]: https://mybinder.org/v2/gh/mbrukman/reimplementing-ml-papers/main?filepath=lenet/LeNet_Keras_v3_Subsamping_fixed_scaling_and_learning_rate_decay.ipynb

In [ ]:
%%bash

# Download the custom Subsampling layer, activation function, and LeNet model.
for module in subsampling activations lenet ; do
  if ! [ -f "${module}.py" ]; then
    curl -sO "https://raw.githubusercontent.com/mbrukman/reimplementing-ml-papers/main/lenet/${module}.py"
  fi
done

In [ ]:
# Local imports downloaded above.
from subsampling import Subsampling
from activations import scaled_tanh
from lenet import LeNet

from tensorflow import keras


# Define the model architecture.
model = LeNet(subsampling=Subsampling, activation=scaled_tanh)

model.summary()

Model: "LeNet-5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 C1 (Conv2D)                 (None, 28, 28, 6)         156       
                                                                 
 S2 (Subsampling)            (None, 14, 14, 6)         12        
                                                                 
 S2_act (Activation)         (None, 14, 14, 6)         0         
                                                                 
 C3 (Conv2D)                 (None, 10, 10, 16)        2416      
                                                                 
 S4 (Subsampling)            (None, 5, 5, 16)          32        
                                                                 
 S4_act (Activation)         (None, 5, 5, 16)          0         
                                                                 
 C5 (Conv2D)                 (None, 1, 1, 120)         4812

In [ ]:
def scheduler(epoch: int, lr: float) -> float:
    if epoch < 2:
        eta = 0.0005
    elif epoch < 5:
        eta = 0.0002
    elif epoch < 8:
        eta = 0.0001
    elif epoch < 12:
        eta = 0.00005
    else:
        eta = 0.00001

    mu = 0.02
    h_kk = 1
    return eta / (mu + h_kk)

lr_callback = keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
# Compile the model with optimizer and loss function.
opt = keras.optimizers.Adam(learning_rate=0.0005)
loss_fn = keras.losses.CategoricalCrossentropy()
model.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])

For details on the MNIST dataset including a data exploration, see [MNIST directory in my repo](https://github.com/mbrukman/reimplementing-ml-papers/tree/main/datasets/mnist).

Here, we will import a shared library to process the MNIST dataset into the format that we need to use below for model training and testing.

In [ ]:
%%bash

# Download our library for processing MNIST dataset.
if ! [ -f "mnist.py" ]; then
  curl -sO https://raw.githubusercontent.com/mbrukman/reimplementing-ml-papers/main/datasets/mnist/mnist.py
fi

In [ ]:
%%capture --no-stderr

import mnist

# This will download the MNIST dataset via the Keras library which outputs data
# to stdout, so we silence it above to avoid extraneous output.
mnist_data = mnist.MNIST()
mnist_input_range = (-0.1, 1.175)

In [ ]:
# Train the model.
#
# In this notebook, we scale the input into the range (-0.1, 1.175) as in the
# paper and convert the labels y to a categorical (one-hot) encoding from the
# default numeric values.
#
# For consistency, we use the same transformations for the test dataset below.
history = model.fit(mnist_data.x_train_scale_custom(mnist_input_range),
                    mnist_data.y_train_categorical(),
                    epochs=20,
                    callbacks=[lr_callback])

Epoch 1/20
1875/1875 [==============================] - 16s 4ms/step - loss: 0.3410 - accuracy: 0.8947 - lr: 4.9020e-04
Epoch 2/20
1875/1875 [==============================] - 7s 3ms/step - loss: 0.1021 - accuracy: 0.9693 - lr: 4.9020e-04
Epoch 3/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0568 - accuracy: 0.9829 - lr: 1.9608e-04
Epoch 4/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0457 - accuracy: 0.9862 - lr: 1.9608e-04
Epoch 5/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0387 - accuracy: 0.9887 - lr: 1.9608e-04
Epoch 6/20
1875/1875 [==============================] - 7s 3ms/step - loss: 0.0300 - accuracy: 0.9916 - lr: 9.8039e-05
Epoch 7/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0268 - accuracy: 0.9923 - lr: 9.8039e-05
Epoch 8/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0243 - accuracy: 0.9931 - lr: 9.8039e-05
Epoch 9/20
1875/1875 [=========================

In [ ]:
# Evaluate the model
#
# Note that we use the same input range scaling and label encoding as above.
model.evaluate(mnist_data.x_test_scale_custom(mnist_input_range),
               mnist_data.y_test_categorical())

313/313 [==============================] - 1s 3ms/step - loss: 0.0380 - accuracy: 0.9879


[0.0379730761051178, 0.9879000186920166]